In [1]:
import json

sweep_config = {
    "method": "grid",
    "metric": {
        "name": "val_accuracy",
        "goal": "maximize"
    },
    "parameters": {
        "dataset": {"value": "CIFAR100"},
        "data_path": {"value": "/kaggle/input/fii-atnn-2024-project-noisy-cifar-100/fii-atnn-2024-project-noisy-cifar-100"},
        "model_name": {"values": [ "resnet18_resize", "resnet50_resize"]},
        "num_classes": {"value": 100},
        "batch_size": {"values": [64]},
        "num_epochs": {"values": [100]},
         # "learning_rate": {
        "optimizer_config": {
        "values": [
            {"optimizer": "adamw", "learning_rate": 0.0005},
             {"optimizer": "adamw", "learning_rate": 0.001},
             {"optimizer": "sgd", "learning_rate": 0.01}
        ]
    },
        "weight_decay": {"value": 0.0005},
        # "optimizer": {"values": ["adamw", "sgd"]},
        "momentum": {"value": 0.9},
        "nesterov": {"value": True},
        "patience": {"value": 3},
        "stop_mode": {"value": "max"},
        "min_delta": {"value": 0.0001},
        "scheduler": {"values": ["cosineannealinglr"]},
        "t_max": {"values": [100]},
        "eta_min": {"value": 0.00001},
        "augmentation_scheme": {"values": ["randaugment", "combined_resize2"]},
        "use_cutmix": {"values": [True]},
        "use_mixup": {"values": [True]},
        "alpha": {"value": 1.0},
        "t_0": {"value": 10},
        "t_mult": {"value": 2},
        "warmup": {"value": 5},
        "patience_early_stopping": {"value": 5},
        "pretrained": {"value": True}
    }
}


with open("sweep_config.json", "w") as f:
    json.dump(sweep_config, f)


In [ ]:
import json

import numpy as np
import yaml
from torch import nn, Tensor
import torch.nn.functional as F
import random

import torch.backends.cudnn


from typing import Literal, cast, Optional, Callable

import torch
import torchvision
from torch import nn
from torchvision.datasets import CIFAR100
from torchvision.transforms.v2 import CutMix, MixUp

from torchvision.transforms import v2
from timm import create_model
import os
import pickle
from torchvision import datasets
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import AutoAugment, AutoAugmentPolicy

import torch.optim.lr_scheduler as lr_scheduler
pin_memory = True

def get_device():
    return "cuda" if torch.cuda.is_available() else "cpu"


def cache_dataset(dataset_class, data_dir, cache_dir='./cache', train=True):
    os.makedirs(cache_dir, exist_ok=True)
    subset = 'train' if train else 'test'
    cache_path = os.path.join(cache_dir, f'{dataset_class.__name__}_{subset}.pkl')

    if os.path.exists(cache_path):
        with open(cache_path, 'rb') as f:
            data = pickle.load(f)
    else:
        data = dataset_class(root=data_dir, train=train, download=True)
        with open(cache_path, 'wb') as f:
            pickle.dump(data, f)

    return data


def get_data_augmentation(scheme="basic", dataset="CIFAR"):
    if dataset == "CIFAR":
        if scheme == "basic":
            train_transform = v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True),
                                          v2.Normalize((0.5, 0.5, 0.5), (0.25, 0.25, 0.25))])
        elif scheme == "random_flip":
            train_transform = v2.Compose(
                [v2.RandomHorizontalFlip(), v2.ToImage(), v2.ToDtype(torch.float32, scale=True),
                 v2.Normalize((0.5, 0.5, 0.5), (0.25, 0.25, 0.25))])
        elif scheme == "random_crop_flip":
            train_transform = v2.Compose([v2.RandomCrop(32, padding=4), v2.RandomHorizontalFlip(), v2.ToImage(),
                                          v2.ToDtype(torch.float32, scale=True),
                                          v2.Normalize((0.5, 0.5, 0.5), (0.25, 0.25, 0.25))])
        elif scheme == "randaugment":
            train_transform = v2.Compose(
                [v2.RandAugment(), v2.ToImage(), v2.ToDtype(torch.float32, scale=True),
                 v2.Normalize((0.5, 0.5, 0.5), (0.25, 0.25, 0.25))])
        elif scheme == "autoaugment":
            train_transform = v2.Compose(
                [AutoAugment(policy=AutoAugmentPolicy.CIFAR10), v2.ToImage(), v2.ToDtype(torch.float32, scale=True),
                 v2.Normalize((0.5, 0.5, 0.5), (0.25, 0.25, 0.25))])

        elif scheme == "combined":
            train_transform = v2.Compose(
                [v2.RandomResizedCrop(32, scale=(0.8, 1.0)), v2.RandomHorizontalFlip(), v2.RandomRotation(15),
                 v2.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1), v2.RandAugment(), v2.ToImage(),
                 v2.ToDtype(torch.float32, scale=True),
                 v2.Normalize((0.5, 0.5, 0.5), (0.25, 0.25, 0.25))])
        elif scheme == "combined2":
            train_transform = v2.Compose(
                [AutoAugment(policy=AutoAugmentPolicy.CIFAR10), v2.RandomCrop(32, padding=4), v2.RandomHorizontalFlip(),
                 v2.ColorJitter(brightness=0.2, contrast=0.2),
                 v2.RandomRotation(15), v2.AutoAugment(), v2.ToImage(), v2.ToDtype(torch.float32, scale=True),
                 v2.Normalize((0.5,), (0.5,))])
        elif scheme == "combined_resize":
            train_transform = v2.Compose([
                v2.Resize((64, 64)),
                v2.RandomResizedCrop(64, scale=(0.8, 1.0)),
                v2.RandomRotation(15),
                v2.RandomHorizontalFlip(),
                v2.RandAugment(),
                v2.ToImage(),
                v2.ToDtype(torch.float32, scale=True),
                v2.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
            ])
        elif scheme == "combined_resize2":
            train_transform = v2.Compose([
                v2.RandomRotation(10),
                v2.RandomResizedCrop(32, scale=(0.9, 1.1)),
                v2.RandomHorizontalFlip(),
                v2.RandomAffine(degrees=0, shear=10),
                v2.RandomCrop(32, padding=3),
                v2.ToImage(),
                v2.ToDtype(torch.float32, scale=True),
                v2.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
            ])
        else:
            raise ValueError(f"Augmentation scheme '{scheme}' not supported for CIFAR.")
        test_transform = v2.Compose(
            [v2.ToImage(), v2.ToDtype(torch.float32, scale=True), v2.Normalize((0.5, 0.5, 0.5), (0.25, 0.25, 0.25))])

    else:
        raise ValueError(f"Dataset '{dataset}' not supported.")

    return train_transform, test_transform

class SimpleCachedDataset(Dataset):
    def __init__(self, dataset):
        # Runtime transforms are not implemented in this simple cached dataset.
        self.data = tuple([x for x in dataset])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, i):
        return self.data[i]


class CIFAR100_noisy_fine(Dataset):
    """
    See https://github.com/UCSC-REAL/cifar-10-100n, https://www.noisylabels.com/ and `Learning with Noisy Labels
    Revisited: A Study Using Real-World Human Annotations`.
    """

    def __init__(
        self, root: str, train: bool, transform: Optional[Callable], download: bool
    ):
        cifar100 = CIFAR100(
            root=root, train=train, transform=transform, download=download
        )
        data, targets = tuple(zip(*cifar100))

        if train:
            noisy_label_file = os.path.join(root, "CIFAR-100-noisy.npz")
            if not os.path.isfile(noisy_label_file):
                raise FileNotFoundError(
                    f"{type(self).__name__} need {noisy_label_file} to be used!"
                )

            noise_file = np.load(noisy_label_file)
            if not np.array_equal(noise_file["clean_label"], targets):
                raise RuntimeError("Clean labels do not match!")
            targets = noise_file["noisy_label"]

        self.data = data
        self.targets = targets

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, i: int):
        return self.data[i], self.targets[i]

def load_data( batch_size=64, scheme="basic", custom_transforms=None):
    if custom_transforms:
        train_transform, test_transform = custom_transforms
    else:
        train_transform, test_transform = get_data_augmentation(scheme=scheme, dataset="CIFAR")

    try:
        train_set = CIFAR100_noisy_fine(
            '/kaggle/input/fii-atnn-2024-project-noisy-cifar-100/fii-atnn-2024-project-noisy-cifar-100', download=False,
            train=True, transform=train_transform)
        test_set = CIFAR100_noisy_fine(
            '/kaggle/input/fii-atnn-2024-project-noisy-cifar-100/fii-atnn-2024-project-noisy-cifar-100', download=False,
            train=False, transform=test_transform)
    except Exception as e:
        print(f"Error loading dataset: {e}")
        raise

    train_set.transform = train_transform
    test_set.transform = test_transform

    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, pin_memory=pin_memory)
    test_loader = DataLoader(test_set, batch_size=500, pin_memory=pin_memory)

    return train_loader, test_loader


def get_model(dataset, model_name, num_classes, input_size=None, hidden_layers=None, pretrained=True):
    if dataset == 'CIFAR10' or dataset == 'CIFAR100':
        if model_name in ['resnet18', 'resnet18_resize']:
            model = create_model("resnet18", pretrained=pretrained, num_classes=num_classes)
        elif model_name in ['resnet50', 'resnet50_resize']:
            model = create_model("resnet50", pretrained=pretrained, num_classes=num_classes)
        elif model_name == 'resnet18_cifar10':
            model = create_model("hf_hub:edadaltocg/resnet18_cifar10", pretrained=False, num_classes=num_classes)
            model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)

        else:
            raise ValueError(
                f"Model '{model_name}' is not supported for CIFAR.")
        if pretrained and model_name in ['resnet18_resize', 'resnet50_resize']:
            model = nn.Sequential(
                nn.Upsample(size=(224, 224), mode='bilinear', align_corners=False),
                model
            )


    else:
        raise ValueError(f"Dataset '{dataset}' is not supported. Choose 'CIFAR10', 'CIFAR100', or 'MNIST'.")

    return model


import torch.optim as optim


def get_optimizer(optimizer_name, model_parameters, lr=0.001, momentum=0.9, weight_decay=0.0, nesterov=True):
    optimizer_name = optimizer_name.lower()

    if optimizer_name == 'sgd':
        return optim.SGD(model_parameters, lr=lr)
    elif optimizer_name == 'sgd_momentum':
        return optim.SGD(model_parameters, lr=lr, momentum=momentum)
    elif optimizer_name == 'sgd_nesterov':
        return optim.SGD(model_parameters, lr=lr, momentum=momentum, nesterov=nesterov)
    elif optimizer_name == 'sgd_weight_decay':
        return optim.SGD(model_parameters, lr=lr, momentum=momentum, weight_decay=weight_decay)
    elif optimizer_name == 'adam':
        return optim.Adam(model_parameters, lr=lr, weight_decay=weight_decay)
    elif optimizer_name == 'adamw':
        return optim.AdamW(model_parameters, lr=lr, weight_decay=weight_decay)
    elif optimizer_name == 'rmsprop':
        return optim.RMSprop(model_parameters, lr=lr, momentum=momentum, weight_decay=weight_decay)
    else:
        raise ValueError(f"Optimizer '{optimizer_name}' not supported.")


def get_scheduler(optimizer, scheduler_name, **kwargs):
    scheduler_name = scheduler_name.lower()

    if scheduler_name == 'steplr':
        return lr_scheduler.StepLR(optimizer, step_size=kwargs.get('step_size', 10), gamma=kwargs.get('gamma', 0.1))

    elif scheduler_name == 'reducelronplateau':
        mode_str = kwargs.get('mode', 'min')
        if mode_str not in ['min', 'max']:
            raise ValueError("Invalid mode for ReduceLROnPlateau: must be 'min' or 'max'")
        mode = cast(Literal["min", "max"], mode_str)
        return lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode=mode,
            factor=kwargs.get('factor', 0.1),
            patience=kwargs.get('patience', 10)
        )

    elif scheduler_name == 'cosineannealinglr':
        return lr_scheduler.CosineAnnealingLR(
            optimizer,
            T_max=kwargs.get('t_max', 50),
            eta_min=kwargs.get('eta_min', 0)
        )

    elif scheduler_name == 'cosineannealingwarmrestarts':
        return lr_scheduler.CosineAnnealingWarmRestarts(
            optimizer,
            T_0=kwargs.get('t_0', 10),
            T_mult=kwargs.get('t_mult', 1),
            eta_min=kwargs.get('eta_min', 0)
        )

    elif scheduler_name == 'exponentiallr':
        return lr_scheduler.ExponentialLR(optimizer, gamma=kwargs.get('gamma', 0.9))

    elif scheduler_name == 'linearlr':
        return lr_scheduler.LinearLR(
            optimizer,
            start_factor=kwargs.get('start_factor', 1.0),
            end_factor=kwargs.get('end_factor', 0.0),
            total_iters=kwargs.get('total_iters', 100)
        )

    elif scheduler_name == 'none':
        return None

    else:
        raise ValueError(f"Scheduler '{scheduler_name}' not supported.")


def early_stopping(current_score, best_score, patience_counter, patience_early_stopping, min_delta=0.0, mode="max"):
    print("heeei", best_score, current_score, mode, patience_counter)
    if best_score is None:
        best_score = current_score
        return False, best_score, patience_counter

    if mode == "min":
        improvement = best_score - current_score > min_delta
    elif mode == "max":
        improvement = current_score - best_score > min_delta
    else:
        raise ValueError("Mode should be 'min' or 'max'")

    if improvement:
        best_score = current_score
        patience_counter = 0
    else:
        patience_counter += 1

    early_stop = patience_counter >= patience_early_stopping
    return early_stop, best_score, patience_counter


@torch.inference_mode()
def validate_model(model, test_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct, total = 0, 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss = running_loss / len(test_loader.dataset)
    val_accuracy = 100 * correct / total
    return val_loss, val_accuracy


from torch.amp import autocast, GradScaler


def train_model(model, train_loader, test_loader, device, num_epochs, optimizer, num_classes, scheduler_mode=None,
                scheduler=None, patience_early_stopping=5, min_delta=0.0, early_stop_mode="max", learning_rate=0.1,
                warmup=0, grad_alpha=1.0, use_cutmix=True, use_mixup=True, alpha=1.0):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    scaler = GradScaler(device)

    best_val_score = None
    best_train_score = None
    patience_counter = 0
    best_val_accuracy = 0.0
    best_train_accuracy = 0.0
    best_val_loss = 100.0
    best_train_loss = 100.0
    wandb.watch(model, log="all", log_freq=10)
    alpha = float(alpha)
    cutmix = v2.CutMix(num_classes=num_classes, alpha=alpha)
    mixup = v2.MixUp(num_classes=num_classes, alpha=alpha)
    cutmix_or_mixup = v2.RandomChoice([cutmix, mixup])
    rand = random.randint(1000, 9999)
    file_path = f"/kaggle/working/best_model_{rand}.pth"
    print("saving", file_path)
    for epoch in range(num_epochs):
        print("Epoch ", epoch)
        model.train()
        running_loss = 0.0
        correct, total = 0, 0

        if epoch < warmup:
            lr_scale = min(1., float(epoch + 1) / warmup)
            for pg in optimizer.param_groups:
                pg['lr'] = learning_rate * lr_scale

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            if use_cutmix and use_mixup:
                inputs, labels = cutmix_or_mixup(inputs, labels)
            elif use_cutmix:
                inputs, labels = cutmix(inputs, labels)
            elif use_mixup:
                inputs, labels = mixup(inputs, labels)

            optimizer.zero_grad()
            with autocast(device):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()

            #             scaler.unscale_(optimizer)
            #             torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=grad_alpha)

            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            if use_cutmix or use_mixup:
                correct += predicted.eq(labels.argmax(dim=1)).sum().item()
            else:
                correct += predicted.eq(labels).sum().item()

        train_loss = running_loss / len(train_loader.dataset)
        train_accuracy = 100 * correct / total

        val_loss, val_accuracy = validate_model(model, test_loader, criterion, device)
        if train_accuracy > best_train_accuracy:
            best_train_accuracy = train_accuracy
        if train_loss < best_train_loss:
            best_train_loss = train_loss
        if val_loss < best_val_loss:
            best_val_loss = val_loss
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            torch.save(model.state_dict(), file_path)
            print(f"Best model saved with accuracy: {best_val_accuracy:.2f}%")
        print(f"Epoch {epoch + 1}/{num_epochs} - ")
        print(f"Train Loss: {train_loss:.4f} - Train Accuracy: {train_accuracy:.2f}% - ")
        print(f"Val Loss: {val_loss:.4f} - Val Accuracy: {val_accuracy:.2f}%")
        wandb.log({
            'epoch': epoch + 1,
            'train_loss': train_loss,
            'train_accuracy': train_accuracy,
            'val_loss': val_loss,
            'val_accuracy': val_accuracy
        })

        val_score = val_loss if early_stop_mode == "min" else val_accuracy
        train_score = train_loss if early_stop_mode == "min" else train_accuracy

       

        if isinstance(scheduler, lr_scheduler.ReduceLROnPlateau):
            if scheduler_mode == "max":
                scheduler.step(val_accuracy)
            elif scheduler_mode == "min":
                scheduler.step(val_loss)
        elif scheduler:
            scheduler.step()

        early_stop, best_train_score, patience_counter = early_stopping(
            current_score=train_score,
            best_score=best_train_score,
            patience_counter=patience_counter,
            patience_early_stopping=patience_early_stopping,
            min_delta=min_delta,
            mode=early_stop_mode
        )

        if early_stop:
            print("Early stopping triggered. Stopping training.")
            break
        # best_val_score =  best_val_loss if early_stop_mode == "min" else  best_val_accuracy
        # best_train_score =  best_train_loss if early_stop_mode == "min" else  best_train_accuracy
    print("Training complete.")


import wandb


def sweep_train():
    wandb.init()
    config = wandb.config
    try:
        train_loader, test_loader = load_data(
            batch_size=config.batch_size,
            scheme=config.augmentation_scheme,
        )

        model = get_model(
            dataset=config.dataset,
            model_name=config.model_name,
            num_classes=config.num_classes
        )
        optimizer_name = config.optimizer_config["optimizer"]
        learning_rate = config.optimizer_config["learning_rate"]

        optimizer = get_optimizer(
            optimizer_name=optimizer_name,
            model_parameters=model.parameters(),
            lr=learning_rate,
            momentum=config.momentum,
            weight_decay=config.weight_decay,
            nesterov=config.nesterov
        )

        scheduler = get_scheduler(
            optimizer=optimizer,
            scheduler_name=config.scheduler,
            t_max=config.get('t_max', 200),
            eta_min=config.get('eta_min', 0),
            step_size=config.get('step_size', 10),
            gamma=config.get('gamma', 0.1),
            patience=config.get('scheduler_patience', 10),
            factor=config.get('factor', 0.1)
        )

        train_model(
            model=model,
            train_loader=train_loader,
            test_loader=test_loader,
            device=get_device(),
            num_epochs=config.num_epochs,
            optimizer=optimizer,
            scheduler=scheduler,
            patience_early_stopping=config.patience_early_stopping,
            min_delta=config.min_delta,
            early_stop_mode=config.stop_mode,
            learning_rate=learning_rate,
            num_classes=config.num_classes,
            use_cutmix=config.use_cutmix,
            use_mixup=config.use_mixup,
            alpha=config.alpha,
            warmup=config.warmup,
        )
    finally:
        wandb.finish()


def load_config(file_path):
    ext = os.path.splitext(file_path)[-1].lower()
    if ext == ".json":
        with open(file_path, 'r') as f:
            config = json.load(f)
    elif ext in {".yaml", ".yml"}:
        with open(file_path, 'r') as f:
            config = yaml.safe_load(f)
    else:
        raise ValueError("Unsupported file format. Use JSON or YAML.")
    return config


config_file_path = "sweep_config.json"
sweep_config = load_config(config_file_path)

sweep_id = wandb.sweep(sweep_config, project="training-cifar100-noisy")
wandb.agent(sweep_id, sweep_train)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: e31mdz1a
Sweep URL: https://wandb.ai/gheorghitastefana-alexandru-ioan-cuza-university-iasi/training-cifar100-noisy/sweeps/e31mdz1a


wandb: Agent Starting Run: 0s0bqid9 with config:
wandb: 	alpha: 1
wandb: 	augmentation_scheme: randaugment
wandb: 	batch_size: 64
wandb: 	data_path: /kaggle/input/fii-atnn-2024-project-noisy-cifar-100/fii-atnn-2024-project-noisy-cifar-100
wandb: 	dataset: CIFAR100
wandb: 	eta_min: 1e-05
wandb: 	min_delta: 0.0001
wandb: 	model_name: resnet18_resize
wandb: 	momentum: 0.9
wandb: 	nesterov: True
wandb: 	num_classes: 100
wandb: 	num_epochs: 100
wandb: 	optimizer_config: {'learning_rate': 0.0005, 'optimizer': 'adamw'}
wandb: 	patience: 3
wandb: 	patience_early_stopping: 5
wandb: 	pretrained: True
wandb: 	scheduler: cosineannealinglr
wandb: 	stop_mode: max
wandb: 	t_0: 10
wandb: 	t_max: 100
wandb: 	t_mult: 2
wandb: 	use_cutmix: True
wandb: 	use_mixup: True
wandb: 	warmup: 5
wandb: 	weight_decay: 0.0005
wandb: Currently logged in as: gheorghitastefana (gheorghitastefana-alexandru-ioan-cuza-university-iasi). Use `wandb login --relogin` to force relogin


model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

saving /kaggle/working/best_model_7423.pth
Epoch  0
Best model saved with accuracy: 25.44%
Epoch 1/100 - 
Train Loss: 4.3464 - Train Accuracy: 7.10% - 
Val Loss: 3.2486 - Val Accuracy: 25.44%
heeei None 7.102 max 0
Epoch  1
Best model saved with accuracy: 53.15%
Epoch 2/100 - 
Train Loss: 3.5610 - Train Accuracy: 24.57% - 
Val Loss: 1.9561 - Val Accuracy: 53.15%
heeei 7.102 24.574 max 0
Epoch  2
